In [1]:
import sleap
import os

input_folder = "D:\Documents\COMP3850-Group23-Ant-posture\datasets"
output_filename = "main.v001.slp"

In [2]:
files = [file for file in os.scandir(input_folder)
         if file.is_file() and file.name.endswith(".slp") and file.name != output_filename]
display([file.name for file in files])

['col218.slp', 'col224.v001.slp', 'col402.v001.slp', 'quel321_1.v001.slp']

In [3]:
# Create an empty labelled dataset to load individual datasets into
combined = sleap.Labels()

In [4]:
for file in files:
    labels = sleap.load_file(file.path)
    _, base_conflicts, new_conflicts = sleap.Labels.complex_merge_between(combined, labels)
    if base_conflicts or new_conflicts:
        raise RuntimeError()

In [5]:
combined

Labels(labeled_frames=81, videos=4, skeletons=1, tracks=0)

In [ ]:
# Print out information for the combined dataset


In [ ]:
# Save combined dataset to the input folder
combined.save(os.path.join(input_folder, output_filename))